# Polars Run

In [1]:
import os
import polars as pl
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [2]:
PATH_DATA = '../../data-testing/'
# PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output3/'

SLA_NAME = 'polars-sla.pkl'
MEM_NAME = 'polars-mem.pkl'


In [3]:
all_sla = {}
all_memory = {}

In [4]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

##     Task1 & Task2- I/O

In [5]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [6]:
for i in range(10):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1 = pl.read_csv(PATH_DATA+fn)
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1.write_csv(TEMP_PATH + 'temp.csv')
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = pl.read_parquet(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.write_parquet(TEMP_PATH + 'temp.parquet')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(3)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1
2
3
4
5
6
7
8
9


In [7]:
all_sla

{'task1': {'csv': {'0': 0.07507119699994291,
   '1': 0.13317349999988437,
   '5': 0.5544521260003421,
   '10': 0.7139456370000516,
   '25': 3.495285318999777,
   '50': 6.891867982999884},
  'prq': {'0': 0.018105353000009927,
   '1': 0.09672830299996349,
   '5': 0.402797370999906,
   '10': 0.5922757479997927,
   '25': 1.0708583320001708,
   '50': 2.0736468330001117,
   '75': 2.7631111459995736,
   '100': 3.229023566000251,
   '130': 3.138907991999986,
   '175': 4.218433176999952}},
 'task2': {'csv': {'0': 0.0657783209999252,
   '1': 0.211144620000141,
   '5': 1.0515136890003305,
   '10': 4.030666144999941,
   '25': 10.240114004000134,
   '50': 20.422804744000132},
  'prq': {'0': 0.06376765699997122,
   '1': 0.4438431630001105,
   '5': 2.176145902999906,
   '10': 8.756232007000108,
   '25': 19.73786589400015,
   '50': 35.83059959399998,
   '75': 45.23838456099975,
   '100': 43.7818696129998,
   '130': 56.8738055609997,
   '175': 76.23245078799982}}}

In [8]:
all_memory

{'task1': {'csv': {'0': 0.10355507200000003,
   '1': 0.216301568,
   '5': 0.9660866559999999,
   '10': 0.9280921599999998,
   '25': 3.210084352000001,
   '50': 6.289076223999999},
  'prq': {'0': 0.01690419199999993,
   '1': 0.09333964800000016,
   '5': 0.4774133759999999,
   '10': 1.1529748479999995,
   '25': 2.7095162879999997,
   '50': 5.215485952000002,
   '75': 7.802765312000005,
   '100': 9.689972736,
   '130': 12.623507456,
   '175': 17.103314944}},
 'task2': {'csv': {'0': 0.07698022400000004,
   '1': 0.003448832000000124,
   '5': -0.0020971519999997135,
   '10': 0.995033088,
   '25': 3.2681902079999983,
   '50': 6.011183103999997},
  'prq': {'0': 0.010805248000000045,
   '1': 0.0012574719999998152,
   '5': 0.002957311999999934,
   '10': 2.7836293120000004,
   '25': 7.1210557439999995,
   '50': 12.257103872000002,
   '75': -48.43825152,
   '100': 0.013008896000000547,
   '130': -0.03634380800000159,
   '175': -0.011243520000000728}}}

In [9]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [10]:
all_df = [pl.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [2,3,4,5,6,7]]
df_right = pl.read_parquet(PATH_DATA+'data_to_join.parquet')
time.sleep(5)

In [11]:
len(all_df)

6

## Task3 - Sorting

In [12]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [13]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.sort(['Date','Amount'],ascending=[True,False])
    dtemp2 = dtemp1.with_columns(
            pl.col('Amount').sort(descending=True),
            pl.col('Date').sort(descending=False),
    )
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


25.02
50.04
75.06
100.08
130.01
175.02


In [14]:
all_sla[n_task]

{'25': 2.139002270999754,
 '50': 3.5219913639998595,
 '75': 4.588792652999928,
 '100': 6.299555884000256,
 '130': 7.852870647000145,
 '175': 10.087494508999953}

In [15]:
all_memory[n_task]

{'25': 0.609947647999995,
 '50': 1.211113472000001,
 '75': 1.829982208000004,
 '100': 2.4119214079999978,
 '130': 3.1351726079999978,
 '175': 4.217257984}

## Task4 - Filtering

In [16]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [17]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    dtemp2 = (dtemp1
                .filter((pl.col('Amount')>300) & (pl.col('Status').is_in(filt1)))
              )
    
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


25.02
50.04
75.06
100.08
130.01
175.02


In [18]:
all_sla[n_task]

{'25': 1.555972182000005,
 '50': 3.2450826630001757,
 '75': 5.054685518000042,
 '100': 6.595324059000177,
 '130': 8.390305525999793,
 '175': 11.158934979999685}

In [19]:
all_memory[n_task]

{'25': 0.8144855040000039,
 '50': 1.608142848,
 '75': 2.3140884479999997,
 '100': 3.1311912960000043,
 '130': 3.930288128000001,
 '175': 5.242494976000003}

## Task5 - Merging

In [20]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [21]:
df_right2 = (df_right.with_columns((pl.col('Date')+'_'+pl.col('ship-service-level')).alias('key_merge'))
             .select('key_merge','amount_avg'))

In [22]:
for dtemp1 in all_df:
    time.sleep(5)
    print(round(len(dtemp1)/1000000,2))
    dtemp1 = dtemp1.with_columns((pl.col('Date')+'_'+pl.col('ship-service-level')).alias('key_merge'))
    dtemp1.drop_in_place('Date')
    dtemp1.drop_in_place('ship-service-level')

    print('waiting')
    time.sleep(7)
    gc.collect()
    time.sleep(4)
    gc.collect()

    print('start merge')
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.join(df_right2,on='key_merge',how='left')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
waiting
start merge
50.04
waiting
start merge
75.06
waiting
start merge
100.08
waiting
start merge
130.01
waiting
start merge
175.02
waiting
start merge


In [23]:
all_sla[n_task]

{'25': 2.1609405859999242,
 '50': 3.5630416879998847,
 '75': 5.5775829170002,
 '100': 8.731901713999832,
 '130': 8.89929458900042,
 '175': 11.132499302999804}

In [24]:
all_memory[n_task]

{'25': 1.6034447360000001,
 '50': 3.182583808000004,
 '75': 4.795387904000002,
 '100': 6.389682176000001,
 '130': 8.281538560000001,
 '175': 11.153240063999995}

## Task6 - udf apply

In [25]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [26]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [27]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    dtemp1 = dtemp1.with_columns(
        pl.col('Amount').apply(fun).alias('Amount2')
    )
    
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
130.01
175.02


In [28]:
all_sla[n_task]

{'25': 104.55906767499982,
 '50': 208.80257985200024,
 '75': 317.5079418610003,
 '100': 420.8575001080003,
 '130': 547.9256991090006,
 '175': 728.0403018369998}

In [29]:
all_memory[n_task]

{'25': 0.20847411199999755,
 '50': 0.428683264,
 '75': 0.5998469119999967,
 '100': 0.807768063999994,
 '130': 1.0439106560000013,
 '175': 1.4109081599999982}

## Task7 - aggregation

In [30]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [31]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [32]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    dtemp1 = dtemp1.groupby(['Date','Status']).agg([pl.col('Amount').mean().alias('amount_mean')])

    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
100.08
130.01
175.02


In [33]:
all_sla[n_task]

{'0': 3.5105229830005555}

In [34]:
all_memory[n_task]

{'0': -0.0748421119999989}

## Save results

In [35]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/polars-mem.pkl']